In [99]:
import pandas as pd

akas_df = pd.read_csv('data/title.akas.tsv.gz', sep='\t', dtype=str, na_values=["\\N"])

title_df = pd.read_csv('data/title.basics.tsv.gz', sep='\t', dtype=str, na_values=["\\N"])

crew_df = pd.read_csv("data/title.crew.tsv.gz", sep="\t", dtype=str, na_values=["\\N"])

principals_df = pd.read_csv('data/title.principals.tsv.gz', sep='\t', dtype=str, na_values=["\\N"])

ratings_df = pd.read_csv("data/title.ratings.tsv.gz", sep="\t", dtype=str, na_values=["\\N"])

cast_names_df = pd.read_csv("data/name.basics.tsv.gz", sep="\t", dtype=str, na_values=["\\N"])

director_names_df = pd.read_csv("data/name.basics.tsv.gz", sep="\t", dtype=str, na_values=["\\N"])

In [100]:
akas_df.drop(columns=['ordering', 'title', 'language', 'types', 'attributes', 'isOriginalTitle'], inplace=True)

In [101]:
title_df = title_df.loc[title_df['titleType'].isin(['movie', 'tvMovie'])]
title_df = title_df.loc[title_df['isAdult'] == '0']
title_df.drop(columns=['titleType','originalTitle', 'isAdult', 'endYear'], inplace=True)

In [102]:
crew_df.drop(columns=['writers'], inplace=True)
crew_df["directors"] = crew_df["directors"].str.split(",")
crew_df = crew_df.explode("directors")[["tconst", "directors"]]

In [103]:
principals_df = principals_df.loc[(principals_df['category'] == 'actor') | (principals_df['category'] == 'actress')]
principals_df.drop(columns=['ordering', 'category', 'job', 'characters'], inplace=True)

In [104]:
cast_names_df.drop(columns=['birthYear', 'deathYear', 'primaryProfession', 'knownForTitles'], inplace=True)
director_names_df.drop(columns=['birthYear', 'deathYear', 'primaryProfession', 'knownForTitles'], inplace=True)

In [105]:
data = pd.merge(akas_df, title_df, left_on='titleId', right_on='tconst', how='right')

In [106]:
data.head()

,titleId,region,tconst,primaryTitle,startYear,runtimeMinutes,genres
0,tt0000009,NaN,tt0000009,Miss Jerry,1894,45,Romance
1,tt0000009,AU,tt0000009,Miss Jerry,1894,45,Romance
2,tt0000009,HU,tt0000009,Miss Jerry,1894,45,Romance
3,tt0000009,US,tt0000009,Miss Jerry,1894,45,Romance
4,tt0000009,DE,tt0000009,Miss Jerry,1894,45,Romance


In [107]:
data = pd.merge(data, crew_df, on='tconst', how='left')

In [108]:
data = pd.merge(data, principals_df, on='tconst', how='left')

In [109]:
data = pd.merge(data, ratings_df, on='tconst', how='left')

In [110]:
data = pd.merge(data, cast_names_df, on='nconst', how='left')

In [111]:
data = data.rename(columns={"primaryName" : "Cast"})
data.drop(columns=['nconst', 'titleId'], inplace=True)

In [112]:
data = pd.merge(data, director_names_df, left_on='directors', right_on='nconst', how='left')

In [113]:
data = data.rename(columns={"primaryName" : "Director"})
data.drop(columns=['nconst', 'directors'], inplace=True)

In [114]:
data = data.dropna()

In [115]:
data = data.groupby(['tconst']).agg({'primaryTitle' : 'first', 'region' : 'first', 'startYear' : 'first', 'runtimeMinutes' : 'first', 'genres' : 'first', 'averageRating' : 'first', 'numVotes' : 'first', 'Cast': lambda x: ', '.join(x), 'Director': lambda x: ', '.join(set(x))}).reset_index()

In [116]:
data = data.rename(columns={"tconst" : "ID", "primaryName" : "Director", "primaryTitle" : "Title", "startYear" : "Year", "runtimeMinutes" : "RunningTime", "region" : "Region", "genres": "Genres", "averageRating" : "Rating", "numVotes" : "Votes"})

In [117]:
data.head()

,ID,Title,Region,Year,RunningTime,Genres,Rating,Votes,Cast,Director
0,tt0000009,Miss Jerry,AU,1894,45,Romance,5.3,222,"Blanche Bayliss, William Courtenay, Chauncey D...",Alexander Black
1,tt0000574,The Story of the Kelly Gang,AU,1906,70,"Action,Adventure,Biography",6.0,976,"Elizabeth Tait, John Tait, Nicholas Brierley, ...",Charles Tait
2,tt0000591,The Prodigal Son,FR,1907,90,Drama,5.6,31,"Georges Wague, Henri Gouget, Christiane Mandel...",Michel Carré
3,tt0000679,The Fairylogue and Radio-Plays,GB,1908,120,"Adventure,Fantasy",5.2,78,"L. Frank Baum, Frank Burns, Frank Burns, Georg...","Francis Boggs, Otis Turner"
4,tt0000941,Locura de amor,ES,1909,45,Drama,4.6,30,"José Argelagués, Joaquín Carrasco, José Durany...","Ricardo de Baños, Alberto Marro"


In [118]:
data = data[['ID', 'Title', 'Year', 'Genres', 'Director', 'Cast', 'RunningTime', 'Region', 'Rating', 'Votes']]

In [119]:
data.head()

,ID,Title,Year,Genres,Director,Cast,RunningTime,Region,Rating,Votes
0,tt0000009,Miss Jerry,1894,Romance,Alexander Black,"Blanche Bayliss, William Courtenay, Chauncey D...",45,AU,5.3,222
1,tt0000574,The Story of the Kelly Gang,1906,"Action,Adventure,Biography",Charles Tait,"Elizabeth Tait, John Tait, Nicholas Brierley, ...",70,AU,6.0,976
2,tt0000591,The Prodigal Son,1907,Drama,Michel Carré,"Georges Wague, Henri Gouget, Christiane Mandel...",90,FR,5.6,31
3,tt0000679,The Fairylogue and Radio-Plays,1908,"Adventure,Fantasy","Francis Boggs, Otis Turner","L. Frank Baum, Frank Burns, Frank Burns, Georg...",120,GB,5.2,78
4,tt0000941,Locura de amor,1909,Drama,"Ricardo de Baños, Alberto Marro","José Argelagués, Joaquín Carrasco, José Durany...",45,ES,4.6,30


In [120]:
data["Director"] = data["Director"].str.replace(", ", ",", regex=False)
data["Cast"] = data["Cast"].str.replace(", ", ",", regex=False)

In [121]:
data.to_csv("data/data2.csv", index=False, sep=";")